# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 1:15PM,258069,10,PRF-53966,Bio-PRF,Released
1,Feb 23 2023 1:15PM,258069,10,PRF-53969,Bio-PRF,Released
2,Feb 23 2023 1:15PM,258069,10,PRF-53972,Bio-PRF,Released
3,Feb 23 2023 1:15PM,258069,10,PRF-53975,Bio-PRF,Released
4,Feb 23 2023 1:15PM,258069,10,PRF-53981,Bio-PRF,Released
5,Feb 23 2023 1:15PM,258069,10,PRF-53985,Bio-PRF,Released
6,Feb 23 2023 1:15PM,258069,10,PRF-53990,Bio-PRF,Released
7,Feb 23 2023 1:07PM,258068,10,PRF-53978,Bio-PRF,Released
8,Feb 23 2023 12:55PM,258061,10,0086427387,ISDIN Corporation,Released
9,Feb 23 2023 12:55PM,258061,10,0086427388,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258062,Released,22
33,258065,Released,1
34,258066,Released,1
35,258068,Released,1
36,258069,Released,7


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258062,NaN,NaN,22.0
258065,NaN,NaN,1.0
258066,NaN,NaN,1.0
258068,NaN,NaN,1.0
258069,NaN,NaN,7.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0.0,0.0,13.0
257943,0.0,0.0,1.0
257983,0.0,0.0,2.0
257986,0.0,0.0,2.0
257988,0.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0,0,13
257943,0,0,1
257983,0,0,2
257986,0,0,2
257988,0,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,0,0,13
1,257943,0,0,1
2,257983,0,0,2
3,257986,0,0,2
4,257988,0,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,,,13
1,257943,,,1
2,257983,,,2
3,257986,,,2
4,257988,,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF
7,Feb 23 2023 1:07PM,258068,10,Bio-PRF
8,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation
23,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation
71,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.
72,Feb 23 2023 12:50PM,258065,10,"Snap Medical Industries, LLC"
73,Feb 23 2023 12:47PM,258056,10,ISDIN Corporation
100,Feb 23 2023 12:46PM,258062,10,"Methapharm, Inc."
122,Feb 23 2023 12:32PM,258055,10,Emerginnova
124,Feb 23 2023 12:22PM,258054,12,HealthCaps LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF,,,7
1,Feb 23 2023 1:07PM,258068,10,Bio-PRF,,,1
2,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation,,,15
3,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation,,,48
4,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.,,,1
5,Feb 23 2023 12:50PM,258065,10,"Snap Medical Industries, LLC",,,1
6,Feb 23 2023 12:47PM,258056,10,ISDIN Corporation,,,27
7,Feb 23 2023 12:46PM,258062,10,"Methapharm, Inc.",,,22
8,Feb 23 2023 12:32PM,258055,10,Emerginnova,,,2
9,Feb 23 2023 12:22PM,258054,12,HealthCaps LLC,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF,7,,
1,Feb 23 2023 1:07PM,258068,10,Bio-PRF,1,,
2,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation,15,,
3,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation,48,,
4,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.,1,,
5,Feb 23 2023 12:50PM,258065,10,"Snap Medical Industries, LLC",1,,
6,Feb 23 2023 12:47PM,258056,10,ISDIN Corporation,27,,
7,Feb 23 2023 12:46PM,258062,10,"Methapharm, Inc.",22,,
8,Feb 23 2023 12:32PM,258055,10,Emerginnova,2,,
9,Feb 23 2023 12:22PM,258054,12,HealthCaps LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF,7,,
1,Feb 23 2023 1:07PM,258068,10,Bio-PRF,1,,
2,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation,15,,
3,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation,48,,
4,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF,7.0,NaN,NaN
1,Feb 23 2023 1:07PM,258068,10,Bio-PRF,1.0,NaN,NaN
2,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation,15.0,NaN,NaN
3,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation,48.0,NaN,NaN
4,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 1:15PM,258069,10,Bio-PRF,7.0,0.0,0.0
1,Feb 23 2023 1:07PM,258068,10,Bio-PRF,1.0,0.0,0.0
2,Feb 23 2023 12:55PM,258061,10,ISDIN Corporation,15.0,0.0,0.0
3,Feb 23 2023 12:55PM,258058,10,ISDIN Corporation,48.0,0.0,0.0
4,Feb 23 2023 12:51PM,258066,10,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5160814,247.0,18.0,0.0
12,516098,1.0,1.0,0.0
15,1031800,18.0,0.0,0.0
16,258010,1.0,0.0,0.0
19,1032139,12.0,32.0,2.0
20,258052,1.0,0.0,0.0
21,258000,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5160814,247.0,18.0,0.0
1,12,516098,1.0,1.0,0.0
2,15,1031800,18.0,0.0,0.0
3,16,258010,1.0,0.0,0.0
4,19,1032139,12.0,32.0,2.0
5,20,258052,1.0,0.0,0.0
6,21,258000,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,247.0,18.0,0.0
1,12,1.0,1.0,0.0
2,15,18.0,0.0,0.0
3,16,1.0,0.0,0.0
4,19,12.0,32.0,2.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,247.0
1,12,Released,1.0
2,15,Released,18.0
3,16,Released,1.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
Executing,18.0,1.0,0.0,0.0,32.0,0.0,0.0
Released,247.0,1.0,18.0,1.0,12.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,18.0,1.0,0.0,0.0,32.0,0.0,0.0
2,Released,247.0,1.0,18.0,1.0,12.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,Executing,18.0,1.0,0.0,0.0,32.0,0.0,0.0
2,Released,247.0,1.0,18.0,1.0,12.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()